# Simulation and Analysis of our Barcelona Pavilion

In [ ]:
import sys; sys.path.append('..')
import numpy as np, elastic_rods
from bending_validation import suppress_stdout
from linkage_vis import LinkageViewer
import deployment_plots

l = elastic_rods.RodLinkage('pav_final190824__flat_opt.obj')
driver=l.centralJoint()

mat = elastic_rods.RodMaterial('Rectangle', 40000, 0.3, [12,8], stiffAxis=elastic_rods.StiffAxis.D1)
l.setMaterial(mat)

l.setPerSegmentRestLength(np.loadtxt('restlen_pav_final190824__flat_opt.txt'))

jdo = l.dofOffsetForJoint(driver)
fixedVars = list(range(jdo, jdo + 6)) # fix rigid motion for a single joint
with suppress_stdout(): elastic_rods.compute_equilibrium(l, fixedVars=fixedVars)
    
deployAngle = np.deg2rad(90)

view = LinkageViewer(l, width=1024)
view.show()

In [ ]:
postoptStressRecorder = deployment_plots.StressRecorder()

In [ ]:
mkdir -p deployment_frames

In [ ]:
from open_linkage import open_linkage
def equilibriumSolver(tgtAngle, l, opts, fv):
    opts.beta = 1e-8
    opts.gradTol = 1e-4
    opts.useIdentityMetric = False
    return elastic_rods.compute_equilibrium(l, tgtAngle, options=opts, fixedVars=fv)
with suppress_stdout(): cr, actuationForces, tgtAngles = open_linkage(l, driver, deployAngle - l.averageJointAngle, 50, view, zPerturbationEpsilon=0, equilibriumSolver=equilibriumSolver,
                                                                      maxNewtonIterationsIntermediate=20, verbose=10, useTargetAngleConstraint=True, outPathFormat='deployment_frames/frame_{}.msh',
                                                                      iterationCallback=postoptStressRecorder.log);

In [ ]:
deployment_plots.energy_plot(tgtAngles, cr)

In [ ]:
deployment_plots.stress_plot(postoptStressRecorder)

## Deployment gap energy analysis
Optional analysis to quantify how "single-DoFy" the pavilion is. This needs to be run immediatley after (re-)running the first cell.

In [ ]:
import deployment_path_analysis
dpa = deployment_path_analysis.deploymentPathAnalysis(l)
deployment_path_analysis.validateEnergyIncrements(l, epsMax=1e-2)

In [ ]:
openingAngles, stiffnessGaps = deployment_path_analysis.stiffnessGapThroughoutDeployment(l, deployAngle, 100)
view.update()

In [ ]:
from matplotlib import pyplot as plt
plt.plot(openingAngles, stiffnessGaps)
plt.ylabel('Relative Stiffness Gap')
plt.xlabel('Opening Angle')
plt.title('Pavilion Deployment Stiffness Gap')
plt.tight_layout()
plt.savefig('stiffness_gap_pavilion.pdf')
plt.show()

In [ ]:
np.min(stiffnessGaps)

In [ ]:
stiffnessGaps[-1]

In [ ]:
m = deployment_path_analysis.deploymentModeViewer(l)

In [ ]:
m.setAmplitude(0.1)

In [ ]:
m.show()

In [ ]:
# Output fabrication data
from linkage_utils import writeRodSegments
writeRodSegments(l,'rodSegments_postdeploy.txt', zeroBasedIndexing=True)
#np.savetxt('restlen_meshID_1935b524-e979-4340-9245-326f69b6eae0.txt',l.getPerSegmentRestLength())

## Extract "free body diagram" for pieces of rods around joints

We consider joints with the greatest bending stress, twisting stress, torque magnitude or force magnitude

In [ ]:
import importlib, structural_analysis
importlib.reload(structural_analysis)
from structural_analysis import Load, isolateRodPieceAtJoint, getLoadOnEdge, freeBodyDiagramReport

stretchingStresses = np.array([s.rod.stretchingStresses() for s in l.segments()])
# Get the (min, max) bending z-stress over the cross-section.
bendingStresses = np.array([s.rod.bendingStresses() for s in l.segments()])
# Get the principal stresses due to the shearing caused by rod torsion.
twistingStresses = np.array([s.rod.twistingStresses() for s in l.segments()])

In [ ]:
np.max(bendingStresses[:,:, 0]), np.max(twistingStresses[:, :]), np.max(stretchingStresses[:, :])

In [ ]:
sojr = structural_analysis.stressesOnJointRegions(l, edgeDist=1)

In [ ]:
def argmax2d(a): return np.unravel_index(a.argmax(), a.shape)
def argmin2d(a): return np.unravel_index(a.argmin(), a.shape)
argmax2d(sojr[0]), argmin2d(sojr[1]), argmax2d(sojr[2])

In [ ]:
freeBodyDiagramReport(l, 50, 0)

In [ ]:
dc = l.segment(0).rod.deformedConfiguration()

In [ ]:
dc.materialFrame[0].d1

In [ ]:
freeBodyDiagramReport(l, 111, 1)

In [ ]:
(np.argmax(np.linalg.norm(l.rivetNetForceAndTorques()[:, 0:3], axis=1)),
 np.argmax(np.linalg.norm(l.rivetNetForceAndTorques()[:, 3:6], axis=1)))

In [ ]:
freeBodyDiagramReport(l, 45, 0)

In [ ]:
freeBodyDiagramReport(l, 105, 0)

## Compare against pre-optimized design

In [ ]:
deployAngle = 1.570800

In [ ]:
l_preopt = elastic_rods.RodLinkage('../examples/data/20190814_145136_linemodel.obj', 20)
driver_preopt = l_preopt.centralJoint()

mat = elastic_rods.RodMaterial('Rectangle', 40000, 0.3, [12,8], stiffAxis=elastic_rods.StiffAxis.D1)
l_preopt.setMaterial(mat)

jdo_preopt = l_preopt.dofOffsetForJoint(driver)
fixedVars_preopt = list(range(jdo_preopt, jdo_preopt + 6)) # fix rigid motion for a single joint
with suppress_stdout(): elastic_rods.restlen_solve(l_preopt)
with suppress_stdout(): elastic_rods.compute_equilibrium(l_preopt, fixedVars=fixedVars_preopt)

view_preopt = LinkageViewer(l_preopt, width=1024)
view_preopt.show()

In [ ]:
preoptStressRecorder = deployment_plots.StressRecorder()
with suppress_stdout(): preopt_cr, preopt_actuationForces, preopt_tgtAngles = open_linkage(l_preopt, driver_preopt, deployAngle - l_preopt.averageJointAngle, 50, view_preopt, zPerturbationEpsilon=0, equilibriumSolver=equilibriumSolver, maxNewtonIterationsIntermediate=20, verbose=10, useTargetAngleConstraint=True, iterationCallback=preoptStressRecorder.log);

In [ ]:
deployment_plots.energy_plot(preopt_tgtAngles, preopt_cr)

In [ ]:
deployment_plots.stress_plot(preoptStressRecorder)

In [ ]:
preoptStressRecorder.actuationAngle[29]

In [ ]:
from matplotlib import pyplot as plt
deployment_plots.bending_stress_comparison_plot(preoptStressRecorder, 'Pre-optimization', postoptStressRecorder, 'Post-optimization')
plt.title('Bending Stress During Deployment')
plt.tight_layout()
plt.savefig('bending_stress.pdf')

In [ ]:
import importlib
importlib.reload(deployment_plots)
deployment_plots.twisting_stress_comparison_plot(preoptStressRecorder, 'Pre-optimization', postoptStressRecorder, 'Post-optimization')

In [ ]:
deployment_plots.energy_comparison_plot(tgtAngles, cr, 'Post-optimization', preopt_tgtAngles, preopt_cr, 'Pre-optimization')

## Analyze Hessian spectrum

In [ ]:
import compute_vibrational_modes
fixedVarsWithoutActuator = fixedVars[:]
lambdas, modes = compute_vibrational_modes.compute_vibrational_modes(l, fixedVars=[], mtype=compute_vibrational_modes.MassMatrixType.FULL, n=16, sigma=-1e-6)

import mode_viewer, importlib
importlib.reload(mode_viewer);
mview = mode_viewer.ModeViewer(l, modes, lambdas, amplitude=5.0)
mview.show()